A análise de variância (ANOVA) pode ser usada em um teste A/B para comparar a média de uma métrica contínua entre duas versões de um experimento, tratando a variável que identifica o grupo (“A” ou “B”) como uma variável categórica. A ANOVA testa se a média da métrica de interesse é estatisticamente igual entre os grupos, avaliando se a diferença observada pode ser atribuída ao efeito da versão testada e não ao acaso. Dessa forma, ela permite verificar se uma mudança (layout, campanha, precificação, redesign, etc.) produziu impacto significativo no comportamento dos usuários.

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("amirmotefaker/ab-testing-dataset")

print("Path to dataset files:", path)

100%|██████████| 2.07k/2.07k [00:00<00:00, 3.37MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/amirmotefaker/ab-testing-dataset/versions/1


In [3]:
import os

os.listdir(path)


['control_group.csv', 'test_group.csv']

In [6]:
df_control = pd.read_csv(path + "/control_group.csv", sep=";")
df_test = pd.read_csv(path + "/test_group.csv", sep=";")

In [7]:
# Adicionar coluna categórica
df_control["group"] = "A"
df_test["group"] = "B"

# Empilhar as duas bases
df = pd.concat([df_control, df_test], ignore_index=True)

# Verificar colunas
print(df.head())
print(df.columns)

      Campaign Name       Date  Spend [USD]  # of Impressions     Reach  \
0  Control Campaign  1.08.2019         2280           82702.0   56930.0   
1  Control Campaign  2.08.2019         1757          121040.0  102513.0   
2  Control Campaign  3.08.2019         2343          131711.0  110862.0   
3  Control Campaign  4.08.2019         1940           72878.0   61235.0   
4  Control Campaign  5.08.2019         1835               NaN       NaN   

   # of Website Clicks  # of Searches  # of View Content  # of Add to Cart  \
0               7016.0         2290.0             2159.0            1819.0   
1               8110.0         2033.0             1841.0            1219.0   
2               6508.0         1737.0             1549.0            1134.0   
3               3065.0         1042.0              982.0            1183.0   
4                  NaN            NaN                NaN               NaN   

   # of Purchase group  
0          618.0     A  
1          511.0     A  
2    

In [8]:
# Remover linhas sem valor de compra
df_clean = df.dropna(subset=["# of Purchase"])

# Modelo ANOVA
model = ols('Q("# of Purchase") ~ C(group)', data=df_clean).fit()
anova_table = sm.stats.anova_lm(model, typ=1)

print(anova_table)


            df        sum_sq       mean_sq         F    PR(>F)
C(group)   1.0  3.587471e+01     35.874713  0.000909  0.976057
Residual  57.0  2.250290e+06  39478.774128       NaN       NaN


Resultado da ANOVA para o teste A/B

O teste ANOVA comparou a média de # of Purchase entre os grupos A (controle) e B (teste). O valor-p obtido foi 0.976, muito acima do limite usual de significância de 0.05. Isso significa que não há evidência estatística de diferença entre os grupos. Em outras palavras, o comportamento médio de compras no grupo de teste e no grupo de controle é praticamente igual. Assim, com base nesses dados, o redesign testado não gerou impacto significativo no número de compras.